In [215]:
import pandas as pd

In [216]:
df470 = pd.read_csv('../scraped-data-2024/c5/SemaineOlympiqueFrancaiseDeVoile2022_470.csv')
dfnacra = pd.read_csv('../scraped-data-2024/c5/SemaineOlympiqueFrancaiseDeVoile2022_nacra.csv')
dfil6 = pd.read_csv('../scraped-data-2024/c5/SemaineOlympiqueFrancaiseDeVoile2022_ilca6.csv')
dfil7 = pd.read_csv('../scraped-data-2024/c5/SemaineOlympiqueFrancaiseDeVoile2022_ilca7.csv')

In [217]:
import re
import unidecode
import numpy as np

def format_nome(nomes):
    nomes_list = str(nomes).strip().split(" ")
    new_nomes_list = list()
    for i in range(len(nomes_list)-1):
        new_nomes_list.append(nomes_list[i])
        if nomes_list[i] == nomes_list[i].title() and nomes_list[i+1] == nomes_list[i+1].upper():
            new_nomes_list.append(', ')
        else:
            new_nomes_list.append(' ')
    new_nomes_list.append(nomes_list[-1])
    new_nome = ''
    for name in new_nomes_list:
        new_nome += name
    
    return new_nome

def process_string(s):
    # Remove espaços em branco extras
    s = str(s).strip()
    
    numbers = re.findall(r'\d+(?:\.\d+)?', s)
    if numbers == []:
        numbers.append(np.nan)
    
    has_parentheses = 1 if '(' in s or ')' in s else 0
    
    letters = ''.join(re.findall(r'[a-zA-Z]', s))
    if letters == '':
        letters = 0
    return numbers, has_parentheses, letters

In [218]:
df470["Nome Competidor"] = df470["Nome Competidor"].apply(lambda x: format_nome(x).strip())
df470["Nome Competidor"] = df470["Nome Competidor"].apply(lambda x: unidecode.unidecode(x).upper())

In [219]:
df470['Descarte'] = df470['Pontuação Regata'].apply(lambda x: process_string(x)[1])
df470['Punição'] = df470['Pontuação Regata'].apply(lambda x: process_string(x)[2])
df470['Pontuação Regata'] = df470['Pontuação Regata'].apply(lambda x: process_string(x)[0][0])

In [220]:
dfnacra["Nome Competidor"] = dfnacra["Nome Competidor"].apply(lambda x: format_nome(x).strip())
dfnacra["Nome Competidor"] = dfnacra["Nome Competidor"].apply(lambda x: unidecode.unidecode(x).upper())

In [221]:
dfnacra['Descarte'] = dfnacra['Pontuação Regata'].apply(lambda x: process_string(x)[1])
dfnacra['Punição'] = dfnacra['Pontuação Regata'].apply(lambda x: process_string(x)[2])
dfnacra['Pontuação Regata'] = dfnacra['Pontuação Regata'].apply(lambda x: process_string(x)[0][0])

In [222]:
dfil6['Nome Competidor'] = dfil6['Nome Competidor'].apply(lambda x: x.split('(')[0])
dfil6['Nome Competidor'] = dfil6['Nome Competidor'].apply(lambda x: unidecode.unidecode(x).upper().strip())

In [223]:
dfil6['Pontuação Regata'] = dfil6['Pontuação Regata'].apply(lambda x: x.replace('\n', ' - '))
dfil6["Pontuação Regata"] = dfil6["Pontuação Regata"].apply(lambda x: x.replace('\xa0', ''))

In [224]:
def devolve_valores(line):
    elements = str(line).strip().split(' - ')
    if elements[0].isnumeric():
        punicao = 0
    else:
        punicao = elements[0]
    
    value = str(elements[1]).strip('(').strip(')')
    if value == elements[1]:
        descarte = 0
    else:
        descarte = 1
    return punicao, descarte, value

In [225]:
dfil6["Punição"] = dfil6["Pontuação Regata"].apply(lambda x: devolve_valores(x)[0])
dfil6["Descarte"] = dfil6["Pontuação Regata"].apply(lambda x: devolve_valores(x)[1])
dfil6["Pontuação Regata"] = dfil6["Pontuação Regata"].apply(lambda x: devolve_valores(x)[2])

In [226]:
dfil7['Nome Competidor'] = dfil7['Nome Competidor'].apply(lambda x: x.split('(')[0])
dfil7['Nome Competidor'] = dfil7['Nome Competidor'].apply(lambda x: unidecode.unidecode(x).upper().strip())

In [227]:
dfil7['Pontuação Regata'] = dfil7['Pontuação Regata'].apply(lambda x: x.replace('\n', ' - '))
dfil7["Pontuação Regata"] = dfil7["Pontuação Regata"].apply(lambda x: x.replace('\xa0', ''))

In [228]:
dfil7["Punição"] = dfil7["Pontuação Regata"].apply(lambda x: devolve_valores(x)[0])
dfil7["Descarte"] = dfil7["Pontuação Regata"].apply(lambda x: devolve_valores(x)[1])
dfil7["Pontuação Regata"] = dfil7["Pontuação Regata"].apply(lambda x: devolve_valores(x)[2])

In [229]:
pd.concat([df470, dfil6, dfil7, dfnacra]).to_csv('../scraped-data-2024/cluster5-pdf.csv', index=False)